# YOLO CAR
2023 08 16  
1540i  
let get init  

In [21]:
import os
current_dir = os.getcwd()  #Here
# List the contents of the "data" directory
train_path = os.path.join(current_dir, "training_images")
data_directory_contents = os.listdir(train_path)
data_directory_contents[:5]

['vid_4_18340.jpg',
 'vid_4_2460.jpg',
 'vid_4_10720.jpg',
 'vid_4_14220.jpg',
 'vid_4_6160.jpg']

In [22]:
import os
current_dir = os.getcwd()  #Here
# List the contents of the "data" directory
test_path = os.path.join(current_dir, "testing_images")
data_directory_contents = os.listdir(test_path)
data_directory_contents[:5]

['vid_5_30040.jpg',
 'vid_5_27500.jpg',
 'vid_5_28380.jpg',
 'vid_5_26420.jpg',
 'vid_5_31160.jpg']

In [ ]:
import pandas as pd

# Load the bounding boxes CSV file
bounding_boxes_csv_path = os.path.join(current_dir, 'train_solution_bounding_boxes.csv')
bounding_boxes_df = pd.read_csv(bounding_boxes_csv_path)

# Display the first few rows of the CSV file
bounding_boxes_df.head()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random

# Define a function to visualize bounding boxes on images
def visualize_bounding_boxes(image_path, bounding_boxes):
    fig, ax = plt.subplots(1, figsize=(12, 9))
    image = Image.open(image_path)
    ax.imshow(image)
    
    for box in bounding_boxes:
        rect = patches.Rectangle(
            (box['xmin'], box['ymin']), 
            box['xmax'] - box['xmin'], 
            box['ymax'] - box['ymin'], 
            linewidth=1, 
            edgecolor='r', 
            facecolor='none'
        )
        ax.add_patch(rect)
    plt.show()

# Randomly select an image from the training set
random_image_name = random.choice(os.listdir(os.path.join(current_dir , 'training_images')))
random_image_path = os.path.join(current_dir, 'training_images', random_image_name)

# Get bounding boxes for the selected image
bounding_boxes = bounding_boxes_df[bounding_boxes_df['image'] == random_image_name].to_dict('records')

# Visualize the image with bounding boxes
visualize_bounding_boxes(random_image_path, bounding_boxes)


In [23]:
import pandas as pd

# Load the bounding boxes CSV file
bounding_boxes_csv_path = os.path.join(current_dir, "train_solution_bounding_boxes.csv")
bounding_boxes_df = pd.read_csv(bounding_boxes_csv_path)

# Display the first few rows of the dataframe
bounding_boxes_df.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


In [24]:
from PIL import Image

def convert_to_yolo_format(row, img_path):
    # Open the image to get its width and height
    with Image.open(img_path) as img:
        img_width, img_height = img.size
    
    # Calculate center, width and height of the bounding box
    center_x = (row['xmin'] + row['xmax']) / 2.0
    center_y = (row['ymin'] + row['ymax']) / 2.0
    width = row['xmax'] - row['xmin']
    height = row['ymax'] - row['ymin']
    
    # Normalize the values
    center_x /= img_width
    center_y /= img_height
    width /= img_width
    height /= img_height
    
    return f"0 {center_x} {center_y} {width} {height}"

# Apply the conversion to each row in the dataframe
training_images_path = os.path.join(current_dir, "training_images")
bounding_boxes_df['yolo_format'] = bounding_boxes_df.apply(lambda row: convert_to_yolo_format(row, os.path.join(training_images_path, row['image'])), axis=1)

# Display the first few rows of the updated dataframe
bounding_boxes_df.head()

,image,xmin,ymin,xmax,ymax,yolo_format
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547,0 0.45043415340236687 0.539816602368421 0.0687...
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180,0 0.10021707670857989 0.5571911197368421 0.155...
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180,0 0.44464544142011836 0.5436776061842105 0.181...
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575,0 0.8332127352071006 0.5314510939473683 0.1975...
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422,0 0.11034732271449704 0.5591216215789474 0.171...


In [25]:
# Directory to save YOLO formatted labels
labels_directory = os.path.join(current_dir, "yolo_labels")
os.makedirs(labels_directory, exist_ok=True)

for _, row in bounding_boxes_df.iterrows():
    # Create a text file for each image with the corresponding YOLO format data
    txt_filename = os.path.splitext(row['image'])[0] + '.txt'
    txt_path = os.path.join(labels_directory, txt_filename)
    with open(txt_path, 'w') as f:
        f.write(row['yolo_format'] + "\n")

# Check the first few created files
created_files = os.listdir(labels_directory)[:5]
created_files


['vid_4_740.txt',
 'vid_4_6260.txt',
 'vid_4_13840.txt',
 'vid_4_15000.txt',
 'vid_4_21400.txt']

In [26]:
# Create a names file for class names
classes_names_path = os.path.join(current_dir, "classes.names")
with open(classes_names_path, 'w') as f:
    f.write("object\n")  # Only one class named "object"

# Create a data file with paths to datasets and class names
data_file_content = f"""
classes = 1
train = {os.path.join(current_dir, "train.txt")}
valid = {os.path.join(current_dir, "valid.txt")}
names = {classes_names_path}
backup = backup/
"""

data_file_path = os.path.join(current_dir, "dataset.data")
with open(data_file_path, 'w') as f:
    f.write(data_file_content)

data_file_path


'/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/dataset.data'

In [27]:
import random

# Get all training image paths
all_image_paths = [os.path.join(training_images_path, img_name) for img_name in bounding_boxes_df['image'].tolist()]

# Shuffle and split the dataset into training and validation sets
random.shuffle(all_image_paths)
split_point = int(0.9 * len(all_image_paths))
train_images = all_image_paths[:split_point]
valid_images = all_image_paths[split_point:]

# Save the paths to train.txt and valid.txt files
train_txt_path = os.path.join(current_dir, "train.txt")
valid_txt_path = os.path.join(current_dir, "valid.txt")

with open(train_txt_path, 'w') as f:
    f.write("\n".join(train_images))

with open(valid_txt_path, 'w') as f:
    f.write("\n".join(valid_images))

train_txt_path, valid_txt_path


('/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/train.txt',
 '/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/valid.txt')

# Image size fix

In [28]:
# Open the first image to get its dimensions
with Image.open(all_image_paths[0]) as img:
    img_width, img_height = img.size

img_width, img_height

(676, 380)

In [29]:
# Calculate the network width and height as multiples of 32
network_width = (img_width + 31) // 32 * 32
network_height = (img_height + 31) // 32 * 32

network_width, network_height

(704, 384)

In [30]:
# YOLOv3 configuration for training
yolov3_config = f"""
[net]
# Testing
batch=4
subdivisions=2
# Training
# batch=64
# subdivisions=16
width={network_width}
height={network_height}
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# ... (Many more layers)

[region]
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
bias_match=1
classes=1
coords=4
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1
"""

# Save the configuration to a file
cfg_path = os.path.join(current_dir, "yolov3_custom.cfg")
with open(cfg_path, 'w') as f:
    f.write(yolov3_config)

cfg_path

'/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/yolov3_custom.cfg'

# Num class (?)

In [31]:
num_classes = 4

# Data processing

In [32]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image


class YOLODataset(Dataset):
    def __init__(self, list_path, img_size=416):
        with open(list_path, 'r') as file:
            self.img_files = file.readlines()
        
        self.label_files = [path.replace('images', 'labels').replace('.png', '.txt').replace('.jpg', '.txt') for path in self.img_files]
        self.img_size = img_size

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, index):
        img_path = self.img_files[index % len(self.img_files)].rstrip()
        img = Image.open(img_path).convert('RGB')
        img = img.resize((self.img_size, self.img_size))
        img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
        img = torch.from_numpy(img).permute(2, 0, 1)  # Convert to (C, H, W) format

        label_path = self.label_files[index % len(self.img_files)].rstrip()
        labels = torch.zeros((0, 5), dtype=torch.float32)  # Default empty label
        if os.path.exists(label_path):
            labels_data = [list(map(float, line.split())) for line in open(label_path)]
            if labels_data:
                labels = torch.tensor(labels_data, dtype=torch.float32)

        return img, labels

# Example usage
dataset = YOLODataset(train_txt_path, img_size=704)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)
next(iter(dataloader))


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/owo/anaconda3/envs/torchenv/lib/python3.11/multiprocessing/spawn.py", line 120, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/owo/anaconda3/envs/torchenv/lib/python3.11/multiprocessing/spawn.py", line 130, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'YOLODataset' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/owo/anaconda3/envs/torchenv/lib/python3.11/multiprocessing/spawn.py", line 120, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/owo/anaconda3/envs/torchenv/lib/python3.11/multiprocessing/spawn.py", line 130, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: DataLoader worker (pid(s) 49906, 49910) exited unexpectedly

# Darknet

In [ ]:
import torch.nn as nn

class DarknetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DarknetBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.1),
            nn.Conv2d(out_channels, in_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.LeakyReLU(0.1)
        )

    def forward(self, x):
        return x + self.layer(x)

class Darknet(nn.Module):
    def __init__(self, num_blocks, num_classes):
        super(Darknet, self).__init__()

        def make_layers(in_channels, out_channels, num_blocks):
            layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, bias=False),
                      nn.BatchNorm2d(out_channels),
                      nn.LeakyReLU(0.1)]
            for _ in range(num_blocks):
                layers.append(DarknetBlock(out_channels, out_channels*2))
            return nn.Sequential(*layers)

        self.layer1 = make_layers(3, 32, num_blocks[0])
        self.layer2 = make_layers(32, 64, num_blocks[1])
        self.layer3 = make_layers(64, 128, num_blocks[2])

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Darknet([1, 2, 4], num_classes=4*(5+num_classes)).to(device)
model


In [ ]:
class YOLOv3(nn.Module):
    def __init__(self, num_classes):
        super(YOLOv3, self).__init__()
        
        # Darknet-53 backbone
        self.backbone = Darknet([1, 2, 8, 8, 4], num_classes)
        
        # Detection heads
        self.head1 = self._make_head(128, num_classes)
        self.head2 = self._make_head(256, num_classes)
        self.head3 = self._make_head(512, num_classes)
        
    def _make_head(self, in_channels, num_classes):
        return nn.Sequential(
            nn.Conv2d(in_channels, in_channels*2, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(in_channels*2),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels*2, 3*(4 + 1 + num_classes), kernel_size=1, stride=1, padding=0)
        )
        
    def forward(self, x):
        x = self.backbone(x)
        
        out1 = self.head1(x)
        out2 = self.head2(x)
        out3 = self.head3(x)
        
        return out1, out2, out3

model = YOLOv3(num_classes=num_classes).to(device)
model


In [ ]:
class YOLOLoss(nn.Module):
    def __init__(self, anchors, num_classes, img_dim):
        super(YOLOLoss, self).__init__()
        self.anchors = anchors
        self.num_anchors = len(anchors)
        self.num_classes = num_classes
        self.bbox_attrs = 5 + num_classes
        self.img_dim = img_dim
        self.ignore_thres = 0.5
        self.lambda_coord = 1

        self.mse_loss = nn.MSELoss()
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        nB = x.size(0)
        nA = self.num_anchors
        nC = self.num_classes
        nG = x.size(2)
        
        stride = self.img_dim / nG

        # Convert anchors to current grid size
        scaled_anchors = [(a_w / stride, a_h / stride) for a_w, a_h in self.anchors]
        scaled_anchors = torch.FloatTensor(scaled_anchors).to(device)

        # Tensors for cuda support
        FloatTensor = torch.cuda.FloatTensor if x.is_cuda else torch.FloatTensor
        LongTensor = torch.cuda.LongTensor if x.is_cuda else torch.LongTensor
        ByteTensor = torch.cuda.ByteTensor if x.is_cuda else torch.ByteTensor

        prediction = (
            x.view(nB, nA, self.bbox_attrs, nG, nG)
            .permute(0, 1, 3, 4, 2)
            .contiguous()
        )

        # Get outputs
        x = torch.sigmoid(prediction[..., 0])  # Center x
        y = torch.sigmoid(prediction[..., 1])  # Center y
        w = prediction[..., 2]  # Width
        h = prediction[..., 3]  # Height
        pred_conf = torch.sigmoid(prediction[..., 4])  # Conf
        pred_cls = torch.sigmoid(prediction[..., 5:])  # Cls pred.

        # Calculate offsets for each grid
        grid_x = torch.arange(nG).repeat(nG, 1).view([1, 1, nG, nG]).type(FloatTensor)
        grid_y = torch.arange(nG).repeat(nG, 1).t().view([1, 1, nG, nG]).type(FloatTensor)
        scaled_anchors = FloatTensor(scaled_anchors)
        anchor_w = scaled_anchors[:, 0:1].view((1, nA, 1, 1))
        anchor_h = scaled_anchors[:, 1:2].view((1, nA, 1, 1))

        # Add offset and scale with anchors
        pred_boxes = FloatTensor(prediction[..., :4].shape)
        pred_boxes[..., 0] = x.data + grid_x
        pred_boxes[..., 1] = y.data + grid_y
        pred_boxes[..., 2] = torch.exp(w.data) * anchor_w
        pred_boxes[..., 3] = torch.exp(h.data) * anchor_h

        # Calculate for all targets
        obj_mask = ByteTensor(nB, nA, nG, nG).fill_(0)
        noobj_mask = ByteTensor(nB, nA, nG, nG).fill_(1)
        class_mask = FloatTensor(nB, nA, nG, nG).fill_(0)
        iou_scores = FloatTensor(nB, nA, nG, nG).fill_(0)
        tx = FloatTensor(nB, nA, nG, nG).fill_(0)
        ty = FloatTensor(nB, nA, nG, nG).fill_(0)
        tw = FloatTensor(nB, nA, nG, nG).fill_(0)
        th = FloatTensor(nB, nA, nG, nG).fill_(0)
        tcls = FloatTensor(nB, nA, nG, nG, nC).fill_(0)

        # Convert to position relative to box
        target_boxes = targets[:, 2:6] * nG
        gxy = target_boxes[:, :2]
        gwh = target_boxes[:, 2:]
        # Get anchors with best iou
        ious = torch.stack([self.bbox_wh_iou(anchor, gwh) for anchor in scaled_anchors])
        best_ious, best_n = ious.max(0)
        # Separate target values
        b, target_labels = targets[:, :2].long().t()
        gx, gy = gxy.t()
        gw, gh = gwh.t()
        gi, gj = gxy.long().t()

        # Set masks
        obj_mask[b, best_n, gj, gi] = 1
        noobj_mask[b, best_n, gj, gi] = 0

        # Set noobj mask to zero where iou exceeds ignore threshold
        for i, anchor_ious in enumerate(ious.t()):
            noobj_mask[b[i], anchor_ious > self.ignore_thres, gj[i], gi[i]] = 0

        # Coordinates
        tx[b, best_n, gj, gi] = gx - gx.floor()
        ty[b, best_n, gj, gi] = gy - gy.floor()
        # Width and height
        tw[b, best_n, gj, gi] = torch.log(gw / scaled_anchors[best_n][:, 0] + 1e-16)
        th[b, best_n, gj, gi] = torch.log(gh / scaled_anchors[best_n][:, 1] + 1e-16)
        # One-hot encoding of label
        tcls[b, best_n, gj, gi, target_labels] = 1

        # Compute some obj_mask values to avoid zero division
        obj_mask = obj_mask.bool()
        noobj_mask = noobj_mask.bool()

        # Loss : Mask outputs to ignore non-existing objects (except with conf. loss)
        # Continue the loss calculation
        loss_x = self.mse_loss(x[obj_mask], tx[obj_mask])
        loss_y = self.mse_loss(y[obj_mask], ty[obj_mask])
        loss_w = self.mse_loss(w[obj_mask], tw[obj_mask])
        loss_h = self.mse_loss(h[obj_mask], th[obj_mask])
        
        loss_conf_obj = self.bce_loss(pred_conf[obj_mask], torch.ones_like(pred_conf[obj_mask]))
        loss_conf_noobj = self.bce_loss(pred_conf[noobj_mask], torch.zeros_like(pred_conf[noobj_mask]))
        loss_conf = loss_conf_obj + loss_conf_noobj
        
        loss_cls = self.ce_loss(pred_cls[obj_mask], torch.argmax(tcls[obj_mask], 1))
        total_loss = loss_x + loss_y + loss_w + loss_h + loss_conf + loss_cls
        
        return total_loss

    def bbox_wh_iou(self, wh1, wh2):
        wh2 = wh2.t()
        w1, h1 = wh1[0], wh1[1]
        w2, h2 = wh2[0], wh2[1]
        inter_area = torch.min(w1, w2) * torch.min(h1, h2)
        union_area = (w1 * h1 + 1e-16) + w2 * h2 - inter_area
        return inter_area / union_area

# Create the loss function
yolo_loss = YOLOLoss(anchors, num_classes, 704).to(device)
yolo_loss

In [ ]:
import torch
from torch.utils.data import DataLoader
from models import Darknet
from utils.datasets import ListDataset
from utils.utils import compute_loss

# Configuration
cfg = '/path/to/yolov3_custom.cfg'
data_config = '/path/to/dataset.data'
img_size = 704
epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model and optimizer
model = Darknet(cfg, img_size=img_size).to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters())

# Get dataloader
train_path = '/path/to/train.txt'
dataset = ListDataset(train_path, img_size=img_size)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=dataset.collate_fn)
import torch.optim as optim

# Model
model = YOLOv3(num_classes=num_classes).to(device)

# Loss function
anchors = [(10,13), (16,30), (33,23), (30,61), (62,45), (59,119), (116,90), (156,198), (373,326)]
yolo_loss = YOLOLoss(anchors, num_classes, 704).to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training parameters
num_epochs = 100
print_every = 10

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for batch_idx, (images, targets) in enumerate(dataloader):
        images, targets = images.to(device), targets.to(device)
        
        # Forward pass
        outputs = model(images)
        
        # Loss
        loss = sum([yolo_loss(output, targets) for output in outputs])
        total_loss += loss.item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Print updates
    if (epoch+1) % print_every == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

print("Training complete.")

